# Physics Informed Neural Networks <br> F1 Car Front Wing Aerodymanics

## PINN

In [1]:
import numpy as np
import pandas as pd

In [2]:
in_filepath = "/Users/ggito/repos/pinns/data/"
points_filename = "front_wing_points_final.csv"
norms_filename = "front_wing_norms_final.csv"

wing_df = pd.read_csv(in_filepath + points_filename)
norm_df = pd.read_csv(in_filepath + norms_filename)

print(wing_df)
print(norm_df)

              x         y         z
0      0.440148  0.073950  0.103123
1      0.713695  0.209429  0.055195
2      0.451790  0.021569  0.052462
3      0.032607  0.154912  0.108069
4      0.750952  0.139930  0.113273
...         ...       ...       ...
19995  0.913177  0.282509  0.051195
19996  0.115440  0.221203  0.028832
19997  0.453917  0.034118  0.052462
19998  0.556022  0.063779  0.101915
19999  0.030382  0.130567  0.011402

[20000 rows x 3 columns]
              x         y         z
0     -1.000000 -0.000293 -0.000429
1     -0.283259 -0.649111 -0.705988
2      0.000000  0.000000 -1.000000
3     -0.974604  0.223384  0.015692
4     -0.325352 -0.933390  0.151428
...         ...       ...       ...
19995 -0.018169 -0.132277  0.991046
19996 -0.991501 -0.031098 -0.126326
19997  0.000000  0.000000 -1.000000
19998  1.000000 -0.000293 -0.000429
19999 -0.730616  0.157085 -0.664473

[20000 rows x 3 columns]


In [3]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import csv

In [4]:
def save_log(log_filepath, total_loss, pde_loss, ic_loss, bc_loss, wing_loss):
  new_row = {
    "total_loss": total_loss.item(),
    "pde_loss": pde_loss.item(),
    "ic_loss": ic_loss.item(),
    "bc_loss": bc_loss.item(),
    "wing_loss": wing_loss.item()
  }

  with open(log_filepath, 'a', newline='') as file:
      
    writer = csv.DictWriter(file, fieldnames=new_row.keys())
    
    file.seek(0, 2)
    if file.tell() == 0:
      writer.writeheader()
    
    writer.writerow(new_row)

In [5]:
def grad(x, y):
  return torch.autograd.grad(x, y, grad_outputs=torch.ones_like(x), create_graph=True, retain_graph=True, only_inputs=True)[0]


In [6]:
class PINN(nn.Module):
  def __init__(self, input_dim, output_dim, hidden_units):
    super(PINN, self).__init__()
    self.layers = nn.ModuleList()
    in_units = input_dim
    for units in hidden_units:
      layer = nn.Linear(in_units, units)
      nn.init.xavier_normal_(layer.weight)  # Apply Xavier initialization
      self.layers.append(layer)
      in_units = units
    output_layer = nn.Linear(in_units, output_dim)
    nn.init.xavier_normal_(output_layer.weight)  # Apply Xavier initialization
    self.layers.append(output_layer)

  def forward(self, input):
    for layer in self.layers[:-1]:
      # output = torch.sigmoid(layer(input))
      output = torch.tanh(layer(input))
      # output = torch.relu(layer(input))
      input = output
    output = self.layers[-1](input)
    return output

  def loss(self, 
        x_f, y_f, z_f, t_f, 
        x0, y0, z0, t0, 
        x_b, y_b, z_b, t_b,
        x_w, y_w, z_w, t_w,
        mu, rho, dt, c1, c2, c3, c4, log_filepath):

    xyzt_combinations = torch.cartesian_prod(x_f.flatten(), y_f.flatten(), z_f.flatten(), t_f.flatten())
    output = self(xyzt_combinations)
    u = output[:, 0]
    v = output[:, 1]
    w = output[:, 2]
    p = output[:, 3]

    u_t = grad(u, t_f)
    u_x = grad(u, x_f)
    u_y = grad(u, y_f)
    u_z = grad(u, z_f)
    u_xx = grad(u_x, x_f)
    u_yy = grad(u_y, y_f)
    u_zz = grad(u_z, z_f)

    v_t = grad(v, t_f)
    v_x = grad(v, x_f)
    v_y = grad(v, y_f)
    v_z = grad(v, z_f)
    v_xx = grad(v_x, x_f)
    v_yy = grad(v_y, y_f)
    v_zz = grad(v_z, z_f)

    w_t = grad(w, t_f)
    w_x = grad(w, x_f)
    w_y = grad(w, y_f)
    w_z = grad(w, z_f)
    w_xx = grad(w_x, x_f)
    w_yy = grad(w_y, y_f)
    w_zz = grad(w_z, z_f)

    p_x = grad(p, x_f)
    p_xx = grad(p_x, x_f)
    p_y = grad(p, y_f)
    p_yy = grad(p_y, y_f)
    p_z = grad(p, z_f)
    p_zz = grad(p_z, z_f)

    # b = rho * ( 1/dt * (u_x + v_y) - u_x**2 - 2*u_y*v_x - v_y**2)

    f1 = u_t + u*u_x + v*u_y + w*u_z + (1/rho) * p_x - mu * (u_xx + u_yy + u_zz)
    f2 = v_t + u*v_x + v*v_y + w*v_z + (1/rho) * p_y - mu * (v_xx + v_yy + v_zz)
    f3 = w_t + u*w_x + v*w_y + w*w_z + (1/rho) * p_z - mu * (w_xx + w_yy + w_zz)
    f3 = u_x + v_y + w_z
    # TODO: add poisson equation & impermeability condition
    # f4 = p_xx + p_yy + p_zz - b

    # Initial condition loss
    output_init = self(torch.cat([x0, y0, z0, t0], dim=1))
    u0_pred = output_init[:, 0]
    v0_pred = output_init[:, 1]
    w0_pred = output_init[:, 2]
    p0_pred = output_init[:, 3]

    # for x > 0 and t = 0 -> u, v, p = 0

    u0_true = torch.zeros_like(u0_pred)
    v0_true = torch.zeros_like(v0_pred)
    w0_true = torch.zeros_like(w0_pred)
    p0_true = torch.ones_like(p0_pred)

    ic_loss_u = torch.mean(torch.square(u0_pred - u0_true))
    ic_loss_v = torch.mean(torch.square(v0_pred - v0_true))
    ic_loss_w = torch.mean(torch.square(w0_pred - w0_true))
    ic_loss_p = torch.mean(torch.square(p0_pred - p0_true))

    # Boundary conditions loss

    xyzt_combinations = torch.cartesian_prod(x_b.flatten(), y_b.flatten(), z_b.flatten(), t_b.flatten())
    output_boundary = self(xyzt_combinations)
    u_b_pred = output_boundary[:, 0]
    v_b_pred = output_boundary[:, 1]
    w_b_pred = output_boundary[:, 2]

    # u = 1, v = 0 and w = 0 for x = 0

    u_b_true = torch.ones_like(u_b_pred)
    v_b_true = torch.zeros_like(v_b_pred)
    w_b_true = torch.zeros_like(w_b_pred)
    
    bc_loss_u = torch.mean(torch.square(u_b_pred - u_b_true))
    bc_loss_v = torch.mean(torch.square(v_b_pred - v_b_true))
    bc_loss_w = torch.mean(torch.square(w_b_pred - w_b_true))

    # Wing surface boundary conditions loss

    xyzt_combinations = torch.cartesian_prod(x_w.flatten(), y_w.flatten(), z_w.flatten(), t_w.flatten())
    output_wing = self(xyzt_combinations)
    u_w_pred = output_wing[:, 0]
    v_w_pred = output_wing[:, 1]
    w_w_pred = output_wing[:, 2]

    u_w_true = torch.zeros_like(u_w_pred)
    v_w_true = torch.zeros_like(v_w_pred)
    w_w_true = torch.zeros_like(w_w_pred)
    
    wing_loss_u = torch.mean(torch.square(u_w_pred - u_w_true))
    wing_loss_v = torch.mean(torch.square(v_w_pred - v_w_true))
    wing_loss_w = torch.mean(torch.square(w_w_pred - w_w_true))

    # Combine PDE residual, initial condition, and boundary condition losses
    pde_loss =  torch.mean(torch.square(f1)) + \
                torch.mean(torch.square(f2)) + \
                torch.mean(torch.square(f3)) / 3
    
    ic_loss = (ic_loss_u + ic_loss_v + ic_loss_p) / 3 
    
    bc_loss = (bc_loss_u + bc_loss_v + bc_loss_w) / 3

    wing_loss = (wing_loss_u + wing_loss_v + wing_loss_w) / 3

    total_loss =  c1 * pde_loss + \
                  c2 * ic_loss + \
                  c3 * bc_loss + \
                  c4 * wing_loss

    save_log(log_filepath, total_loss, pde_loss, ic_loss, bc_loss, wing_loss)

    return total_loss, pde_loss, ic_loss, bc_loss, wing_loss

In [7]:
x_max = 1
y_max = 1
z_max = 1
t_max = 1

Nx = 10
Ny = 10
Nz = 10
Nt = 10

dx = x_max / (Nx - 1)
dy = y_max / (Ny - 1)
dz = z_max / (Nz - 1)
dt = t_max / (Nt - 1)

x_test = np.linspace(0, x_max, Nx)
y_test = np.linspace(0, y_max, Ny)
z_test = np.linspace(0, z_max, Nz)
t_test = np.linspace(0, t_max, Nt)

x_grid, y_grid, z_gripd, t_grid = np.meshgrid(x_test, y_test, z_test, t_test)

In [8]:
input_dim = 4
output_dim = 4
# hidden_units = [32, 32, 32]
# hidden_units = [64, 64, 64, 64]
# hidden_units = [128, 128, 128, 128]
# hidden_units = [256, 256, 256, 256]
# hidden_units = [512, 512]
hidden_units = [1024, 1024, 1024]
# hidden_units = [20, 40, 80, 100, 100, 80, 40, 20]

if torch.backends.mps.is_available():
  device = torch.device("mps")
  x = torch.ones(1, device=device)
  print (x)
else:
  print ("MPS device not found.")
  device = "cpu"

# device = "cpu"

pinn = PINN(input_dim, output_dim, hidden_units).to(device)

tensor([1.], device='mps:0')


In [9]:
# optimizer = torch.optim.Adam(pinn.parameters(), lr=0.001)
optimizer = torch.optim.LBFGS(pinn.parameters())

epochs = 1000
Nf = Nx   # num of collocation points -> pde evaluation -> Nf^4... needs fixing: sample Nf points from the whole 3D domain
N0 = Ny   # num of points to evaluate initial conditons -> N0^4
Nb = Nx   # num of points to evaluate boundary conditions -> Nb^4
Nw = Nx   # num of points of the surface of the front wing to evaluate boundary conditions -> Nw^4

# Density (rho): 1.184 kg/m³
# Dynamic viscosity (mu): 1.81e-5 kg/m.s
rho = 1184
mu = 1.81e-5

log_filepath = "/Users/ggito/repos/pinns/data/log.csv"

In [10]:
def sample_points_in_domain(min, max, num_of_samples):
  return torch.tensor(np.random.uniform(min, max, size=(num_of_samples, 1)), dtype=torch.float32, device=device, requires_grad=True)

In [11]:
def zeros(num):
  return torch.zeros(size=(num, 1), dtype=torch.float32, device=device, requires_grad=True)

In [12]:
def closure(report_losses=False):
  optimizer.zero_grad()

  # TODO: use quasi monte carlo sampling
  z_f = sample_points_in_domain(0, x_max, Nf)
  y_f = sample_points_in_domain(0, y_max, Nf)
  x_f = sample_points_in_domain(0, z_max, Nf)
  t_f = sample_points_in_domain(0, t_max, Nf)

  x0 = sample_points_in_domain(dx, x_max, N0)
  y0 = sample_points_in_domain(dy, y_max, N0)
  z0 = sample_points_in_domain(dz, z_max, N0)
  t0 = sample_points_in_domain(0, t_max, N0)

  x_b = zeros(Nb)
  y_b = sample_points_in_domain(0, y_max, Nb)
  z_b = sample_points_in_domain(0, z_max, Nb)
  t_b = sample_points_in_domain(0, t_max, Nb)
  
  # sample Nw wing points
  sampled_indices = wing_df.sample(n=Nw).index

  x_w, y_w, z_w = [torch.tensor(wing_df.loc[sampled_indices, col].values, dtype=torch.float32, device=device, requires_grad=True) for col in ['x', 'y', 'z']]
  t_w = sample_points_in_domain(0, t_max, Nw)

  total_loss, pde_loss, ic_loss, bc_loss, wing_loss = pinn.loss(
                  x_f, y_f, z_f, t_f,
                  x0, y0, z0, t0,
                  x_b, y_b, z_b, t_b,
                  x_w, y_w, z_w, t_w,
                  mu, rho, dt,
                  0.005, 0.25, 0.25, 0.25,
                  log_filepath)

  total_loss.backward()

  if report_losses:
      return total_loss, pde_loss, ic_loss, bc_loss, wing_loss
  else:
      return total_loss

for epoch in range(epochs):
  optimizer.step(closure)
  # if epoch % 10 == 0:
  total_loss, pde_loss, ic_loss, bc_loss, wing_loss = closure(report_losses=True)
  print(f'Epoch: {epoch},\tTotal loss: {total_loss.item()},\tPDE loss: {pde_loss.item()},\tIC loss: {ic_loss.item()},\tBC loss: {bc_loss.item()},\tWing loss: {wing_loss.item()}')

KeyboardInterrupt: 